In [2]:
#%%
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from modelCore import build_core_model
from load_data import load_data_func
from keras import callbacks
import increase_lr
from tensorflow import keras
import one_cycle_scheduler
from tensorflow.keras.optimizers import Adam

# Define input shape and number of classes
input_shape = (320, 320, 3)
# # Load the pre-trained model
# pretrained_model_path = 'face_recognition_core_model.h5'  # Adjust the path to your pre-trained model
# pretrained_model = tf.keras.models.load_model(pretrained_model_path)

# # Compile the pre-trained model
# pretrained_model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Load the augmented face images and labels
dataset_dir = '../final_project/dataset/augmented_dataset'
images, labels, label_name = load_data_func(dataset_dir)

# Shuffle the images and labels
indices = np.arange(len(images))
np.random.shuffle(indices)
images = images[indices]
labels = labels[indices]

# Build the core model
core_model = build_core_model(input_shape, len(label_name))
# # Transfer the weights from the pre-trained model to the core model
# core_model.set_weights(pretrained_model.get_weights())

# Split the data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Compile the core model with SGD optimizer and momentum
optimizer = keras.optimizers.Adam(learning_rate=0.01)
core_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define one-cycle learning rate scheduler
num_samples = len(train_images)
max_lr = 0.1
batch_size = 32
num_epochs = 10
pct_start = 0.3
anneal_strategy = 'cos'
one_cycle_scheduler_cb = one_cycle_scheduler.OneCycleLr(num_samples, num_epochs, max_lr, batch_size, pct_start, anneal_strategy)
stopper = callbacks.EarlyStopping(monitor='val_loss', patience=3)
stop_learing_rate = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

# Define checkpoint callback to save the best model
checkpoint = callbacks.ModelCheckpoint('../final_project/checkpoint/model-{epoch:03d}.keras', monitor='val_loss', verbose=0, save_best_only=True, mode='auto')

# Normalize pixel values to [0, 1]
images = images / 255.0

# Train the model with callbacks
history = core_model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(val_images, val_labels), callbacks=[checkpoint, one_cycle_scheduler_cb, stopper, stop_learing_rate])

# Evaluate the model
test_loss, test_accuracy = core_model.evaluate(val_images, val_labels)
print(f'Test accuracy: {test_accuracy}')

# Save the trained model
core_model.save('face_recognition_core_model.h5')

# Visualize training history
from matplotlib import pyplot as plt

plt.plot(history.history['loss'], 'r', label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], 'r', label='training accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.xlabel('# epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluate the model
test_loss, test_accuracy = core_model.evaluate(val_images, val_labels)
print(f'Test accuracy: {test_accuracy}')
 
# %%


Epoch 1/10


ValueError: Exception encountered when calling MaxPooling2D.call().

[1mNegative dimension size caused by subtracting 2 from 1 for '{{node sequential_1_1/max_pooling2d_18_1/MaxPool2d}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](sequential_1_1/batch_normalization_18_1/batchnorm/add_1)' with input shapes: [?,1,1,512].[0m

Arguments received by MaxPooling2D.call():
  • inputs=tf.Tensor(shape=(None, 1, 1, 512), dtype=float32)

In [16]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import datetime
from load_data import load_data_func

model = load_model('face_recognition_core_model.h5')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source = cv2.VideoCapture(0)

image, labels, label_names = load_data_func('dataset/augmented_dataset')

def MarkAttendance(name):
    with open('Attendence.csv','r+') as f:
        myDataList=f.readlines()
        nameList=[]
        for line in myDataList:
            entry=line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now=datetime.datetime.now()
            dtString=now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

while (True):

    ret, img = source.read()
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # Convert RGB to grayscale for face detection
    faces = face_cascade.detectMultiScale(gray, 1.5, 5)

    for (x, y, w, h) in faces:
        face_img = rgb_img[y:y + h, x:x + w]  # Extract face region from RGB image
        resized = cv2.resize(face_img, (320, 320))  # Resize face image to match model input shape
        normalized = resized / 255.0  # Normalize pixel values
        reshaped = np.expand_dims(normalized, axis=0)  # Add batch dimension
        result = model.predict(reshaped)

        label = np.argmax(result, axis=1)[0]

        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(img, label_names[label], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        MarkAttendance(label_names[label])

    cv2.imshow('LIVE', img)
    key = cv2.waitKey(1)

    if (key == 27):
        break

cv2.destroyAllWindows()
source.release()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 574ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 